In [1]:
import re
import json
import pickle
import pandas as pd
from pprint import pprint
from indra_db.util import get_db, get_ro, get_primary_db
from indra_db.util.helpers import unpack
from sqlalchemy import text

/adeft/.virtualenvs/adeft_env/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
db = get_db('primary')

In [3]:
show_tables = """SELECT *
                 FROM
                     pg_catalog.pg_tables
                 WHERE
                     schemaname != 'pg_catalog'
                 AND schemaname != 'information_schema';
              """

In [4]:
res = db.session.execute(show_tables)

In [5]:
list(res)

[('public', 'raw_muts', 'tester', None, True, False, True, False),
 ('public', 'rejected_statements', 'tester', None, True, False, True, False),
 ('public', 'reading_updates', 'tester', None, True, False, False, False),
 ('public', 'raw_agents', 'tester', None, True, False, True, False),
 ('public', 'raw_activity', 'tester', None, True, False, True, False),
 ('public', 'pa_statements', 'tester', None, True, False, True, False),
 ('public', 'pa_agents', 'tester', None, True, False, True, False),
 ('public', 'source_file', 'tester', None, True, False, False, False),
 ('public', 'raw_mods', 'tester', None, True, False, True, False),
 ('public', 'updates', 'tester', None, True, False, False, False),
 ('public', 'raw_unique_links', 'tester', None, True, False, True, False),
 ('public', 'preassembly_updates', 'tester', None, True, False, False, False),
 ('public', 'pa_mods', 'tester', None, True, False, True, False),
 ('public', 'pa_muts', 'tester', None, True, False, True, False),
 ('public

In [4]:
def describe_table(tablename):
    query = """
            SELECT
                column_name, data_type
            FROM
                information_schema.columns
            WHERE
                table_name = :tablename;
            """
    res = db.session.execute(query, {'tablename': tablename})
    return list(res)

In [5]:
describe_table('text_ref')

[('id', 'integer'),
 ('pmid', 'character varying'),
 ('pmid_num', 'integer'),
 ('pmcid', 'character varying'),
 ('pmcid_num', 'integer'),
 ('pmcid_version', 'integer'),
 ('doi', 'character varying'),
 ('doi_ns', 'integer'),
 ('doi_id', 'character varying'),
 ('pii', 'character varying'),
 ('url', 'character varying'),
 ('manuscript_id', 'character varying'),
 ('create_date', 'timestamp without time zone'),
 ('last_updated', 'timestamp without time zone')]

In [8]:
describe_table('text_content')

[('text_ref_id', 'integer'),
 ('source', 'character varying'),
 ('format', 'character varying'),
 ('text_type', 'character varying'),
 ('insert_date', 'timestamp without time zone'),
 ('last_updated', 'timestamp without time zone'),
 ('id', 'integer'),
 ('content', 'bytea'),
 ('preprint', 'boolean')]

This query does a self join on the raw agents table to find texts grounded to each HGNC gene along with statement id's for these texts

In [6]:
def show_indices(tablename):
    query = """
            SELECT
                indexname, indexdef
            FROM
                pg_indexes
            WHERE
                tablename = :tablename;
            """
    res = db.session.execute(query, {'tablename': tablename})
    return list(res)

In [7]:
def show_matviews():
    query = """
        SELECT schemaname AS schema_name,
            matviewname AS view_name,
            matviewowner AS owner,
            ispopulated AS is_populated,
            definition
        FROM pg_matviews
        ORDER BY schema_name,
                 view_name;
        """
    res = db.session.execute(query)
    return list(res)

In [8]:
show_indices('raw_stmt_ref_link')

[]

In [ ]:
query = """
        SELECT
            rs.id, text_ref.id
        FROM
            (SELECT
                id, reading_id
             FROM
                raw_statements
             WHERE
                 id IN (193, 2993, 28283, 29333, 28388)
             ) rs 
        JOIN
            reading
        ON
            rs.reading_id = reading.id
        JOIN
            text_content
        ON
            reading.text_content_id = text_content.id
        JOIN
            text_ref
        ON
            text_content.text_ref_id = text_ref.id
""" 

In [11]:
show_matviews()

[('public', 'pa_stmt_src', 'tester', True, " SELECT final_result.mk_hash,\n    final_result.reach,\n    final_result.phosphosite,\n    final_result.pc11,\n    final_result.hprd,\n    final_resu ... (613 characters truncated) ... t, medscan bigint, trrust bigint, signor bigint, sparser bigint, rlimsp bigint, cbn bigint, tas bigint, bel_lc bigint, biogrid bigint, trips bigint);"),
 ('public', 'raw_stmt_ref_link', 'tester', True, ' SELECT raw_statements.id AS stmt_id,\n    text_ref_old.id AS text_ref_id\n   FROM (((raw_statements\n     JOIN reading ON ((raw_statements.reading_i ... (18 characters truncated) ... n     JOIN text_content ON ((reading.text_content_id = text_content.id)))\n     JOIN text_ref_old ON ((text_content.text_ref_id = text_ref_old.id)));'),
 ('public', 'raw_stmt_src', 'tester', True, ' SELECT raw_statements.id AS sid,\n    lower((reading.reader)::text) AS src\n   FROM raw_statements,\n    reading\n  WHERE (reading.id = raw_statemen ... (39 characters truncated) ... em

In [12]:
query = """REFRESH MATERIALIZED VIEW
               raw_stmt_ref_link
        """
        


In [ ]:
res

In [ ]:
query = """CREATE MATERIALIZED VIEW
               raw_stmt_ref_link
           AS
               SELECT
                   raw_statements.id stmt_id, text_ref.id text_ref_id
               FROM
                   raw_statements
               JOIN
                   reading
               ON
                   raw_statements.reading_id = reading.id
               JOIN
                   text_content
               ON
                   reading.text_content_id = text_content.id
               JOIN
                   text_ref
               ON
                   text_content.text_ref_id = text_ref.id
        """

In [13]:
db.session.execute(query)

In [14]:
db.session.commit()

In [15]:
show_matviews()

[('public', 'pa_stmt_src', 'tester', True, " SELECT final_result.mk_hash,\n    final_result.reach,\n    final_result.phosphosite,\n    final_result.pc11,\n    final_result.hprd,\n    final_resu ... (613 characters truncated) ... t, medscan bigint, trrust bigint, signor bigint, sparser bigint, rlimsp bigint, cbn bigint, tas bigint, bel_lc bigint, biogrid bigint, trips bigint);"),
 ('public', 'raw_stmt_ref_link', 'tester', True, ' SELECT raw_statements.id AS stmt_id,\n    text_ref_old.id AS text_ref_id\n   FROM (((raw_statements\n     JOIN reading ON ((raw_statements.reading_i ... (18 characters truncated) ... n     JOIN text_content ON ((reading.text_content_id = text_content.id)))\n     JOIN text_ref_old ON ((text_content.text_ref_id = text_ref_old.id)));'),
 ('public', 'raw_stmt_src', 'tester', True, ' SELECT raw_statements.id AS sid,\n    lower((reading.reader)::text) AS src\n   FROM raw_statements,\n    reading\n  WHERE (reading.id = raw_statemen ... (39 characters truncated) ... em

In [ ]:
query = """
        SELECT 
            temp_files AS "Temporary files",
            pg_size_pretty(temp_bytes) AS "Size of temporary files"
        FROM   pg_stat_database db;
"""
res = list(db.session.execute(query))

In [ ]:
query = """
        SELECT
            text_ref_id
        FROM
            rstmt_ref_link
        WHERE
            stmt_id IN (32822, 383838)
        """
res = list(db.session.execute(query))

In [ ]:
res

In [ ]:
res[2][1]* 10**(-9)

In [ ]:
views[-2][4]

In [ ]:
show_indices('text_content')

In [ ]:
describe_table('text_content')

In [ ]:
describe_table('text_ref')

In [ ]:
describe_table('source_file')

In [ ]:
db.session.rollback()

In [7]:
show_indices('raw_agents')

[('raw_agents_stmt_id_ag_num_index', 'CREATE INDEX raw_agents_stmt_id_ag_num_index ON public.raw_agents USING btree (stmt_id, ag_num)'),
 ('fki_raw_agents_stmt_id_fkey', 'CREATE INDEX fki_raw_agents_stmt_id_fkey ON public.raw_agents USING btree (stmt_id)'),
 ('raw_agents_db_id_idx', 'CREATE INDEX raw_agents_db_id_idx ON public.raw_agents USING btree (db_id COLLATE "en_US.utf8" varchar_ops)'),
 ('raw_agents_db_name_idx', 'CREATE INDEX raw_agents_db_name_idx ON public.raw_agents USING btree (db_name COLLATE "en_US.utf8" varchar_ops)'),
 ('raw_agents_pkey', 'CREATE UNIQUE INDEX raw_agents_pkey ON public.raw_agents USING btree (id)')]

In [14]:
query = """CREATE INDEX
               raw_agents_stmt_id_ag_num_index
           ON
               raw_agents
           USING
               btree (stmt_id, ag_num)
        """

In [15]:
db.session.execute(query)

In [22]:
db.session.rollback()

In [ ]:
query = """
        SELECT x.stmt_id, x.ag_num, x.db_id, y.db_id, w.reader
        FROM
            raw_agents x
        INNER JOIN
            raw_agents y
        ON x.stmt_id = y.stmt_id AND x.ag_num = y.ag_num
        INNER JOIN
            raw_statements z
        ON x.stmt_id = z.id
        INNER JOIN
            reading w
        ON z.reading_id = w.id
        WHERE
            x.db_name = 'TEXT' AND y.db_name = 'HGNC'
        AND x.stmt_id IS NOT NULL
        """
text2grounding = list(db.session.execute(query))

The output of this query has been pickled so we don't need to run it multiple times

In [9]:
query = """
        SELECT x.stmt_id, x.ag_num, x.db_id, y.db_id
        FROM
            raw_agents x
        INNER JOIN
            raw_agents y
        ON x.stmt_id = y.stmt_id and x.ag_num = y.ag_num
        INNER JOIN
            raw_statements z
        ON x.stmt_id = z.id
        WHERE
            x.db_name = 'TEXT' AND y.db_name = 'HGNC' AND x.stmt_id IS NOT NULL
        """

In [ ]:
text2grounding = list(db.session.execute(query))

In [ ]:
with open('text2grounding.pkl', 'wb') as f:
    pickle.dump(text2grounding, f)

In [ ]:
text2grounding[2]

In [3]:
with open('text2grounding.pkl', 'rb') as f:
    text2grounding = pickle.load(f)

In [4]:
from collections import defaultdict
from indra.databases.hgnc_client import get_hgnc_name

In [5]:
hgnc_texts = defaultdict(lambda: defaultdict(set))

In [6]:
bad = []
for stmt_id, _, text, hgnc_id, reader in text2grounding:
    name = get_hgnc_name(hgnc_id)
    if name:
        hgnc_texts[name][text].add((stmt_id, reader))
    else:
        bad.append(hgnc_id)

In [ ]:
pattern = r'^[A-Z][A-Z0-9]{1,4}$'

In [33]:
s = {gene_name: {text: len(value) for text, value in entry.items() if re.match(pattern, text)}
     for gene_name, entry in hgnc_texts.items()}

In [36]:
a = defaultdict(int)

In [37]:
for gene_name, entry in s.items():
    for text, size in entry.items():
        a[text] += size

In [40]:
frequent_gene_texts = sorted(a.items(), key=lambda x: -x[1])

In [42]:
top = frequent_gene_texts[:1000]

In [43]:
top

[('STAT3', 94539),
 ('VEGF', 91761),
 ('EGFR', 89811),
 ('ER', 86275),
 ('CD4', 80034),
 ('EMT', 61518),
 ('AR', 52607),
 ('TNF', 51221),
 ('EGF', 50195),
 ('TLR4', 45492),
 ('TCR', 40790),
 ('PTEN', 37172),
 ('IR', 35317),
 ('TRAIL', 34767),
 ('BDNF', 32653),
 ('SIRT1', 31235),
 ('MAP', 29849),
 ('ATM', 29206),
 ('STAT1', 28664),
 ('ECM', 25576),
 ('EC', 25150),
 ('CXCR4', 24548),
 ('RANKL', 23925),
 ('HMGB1', 23759),
 ('MDM2', 23471),
 ('GH', 22619),
 ('NLRP3', 21570),
 ('NGF', 21323),
 ('APC', 21282),
 ('GR', 20493),
 ('HGF', 20436),
 ('APP', 20177),
 ('GC', 20114),
 ('FAK', 20055),
 ('HR', 20054),
 ('BRCA1', 20023),
 ('HER2', 19863),
 ('TLR2', 19759),
 ('NAC', 19757),
 ('PCNA', 19633),
 ('PA', 19563),
 ('CD44', 18535),
 ('TG', 17651),
 ('CRP', 17163),
 ('PTH', 16974),
 ('EZH2', 16809),
 ('CD40', 16687),
 ('TPA', 16149),
 ('GA', 15554),
 ('PC', 15247),
 ('ACTH', 14414),
 ('MYC', 14368),
 ('SP', 14260),
 ('TRPV1', 14221),
 ('H2AX', 14176),
 ('CBP', 14096),
 ('CFTR', 14052),
 ('JAK2',

In [44]:
with open('../data/top_gene_texts.pkl', 'wb') as f:
    pickle.dump(top, f)

In [55]:
with open('../data/filenames.json') as f:
    big_map = json.load(f)

In [56]:
top_genes = [x[0] for x in top]

In [57]:
sample_map = {key: value for key, value in big_map.items() if key in top_genes}

In [59]:
with open('../data/sample_map.json', 'w') as f:
    json.dump(sample_map, f)

In [60]:
import subprocess

In [67]:
for filename in sample_map.values():
    subprocess.call(['scp', f'adeft_content:/content/adeft_mining_results/merged_results/miners/{filename}',
                    '../data/miners/'])

In [68]:
len(sample_map)

631

In [ ]:
import nltk
from nltk.corpus import words

In [ ]:
wordset = set(words.words())

Get all genes with grounded from a common english word. Take only texts with over 10 statements. Calculate ratio of statements that came from a common english word

In [ ]:
word_texts = {}
for gene, info in hgnc_texts.items():
    total_stmt_count = sum([len(stmts) for stmts in info.values()])
    filtered_info = {text: stmts for text, stmts in info.items() if len(stmts) > 10}
    texts = set(filtered_info)
    overlap = texts & wordset
    if overlap:
        count4word_texts = sum([len(stmts) for text, stmts in filtered_info.items() if text in overlap])
        word_texts[gene] = (filtered_info, tuple(overlap), count4word_texts/total_stmt_count,
                           total_stmt_count)

In [ ]:
worst_words = sorted(word_texts.items(), key=lambda x: -x[1][2])

In [ ]:
rows = []
for gene, info in worst_words:
    new_row = [gene, ','.join(info[1]), info[2], info[3]]
    rows.append(new_row)
df = pd.DataFrame(rows, columns=['gene', 'english_words', 'proportion', 'total_stmts'])

In [ ]:
df[df.proportion > 0.8].to_csv('worst_english_words.tsv', sep='\t', index=False)

In [ ]:
from indra_db.util.content_scripts import get_text_content_from_stmt_ids
from indra.literature.adeft_tools import get_text_content_for_gene, universal_extract_text

In [ ]:
THG1L_texts = get_text_content_for_gene('THG1L')

In [ ]:
texts = [universal_extract_text(text) for text in THG1L_texts]

In [ ]:
from sklearn.svm import OneClassSVM
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
pipeline = Pipeline([('tfidf',
                    TfidfVectorizer(ngram_range=(1, 2),
                    max_features=10000,
                    stop_words='english')),
                    ('osvm', OneClassSVM(gamma='scale'))])                  

In [ ]:
params = {'tfidf__max_features': [100, 1000],
          'tfidf__ngram_range': [(1, 1), (1, 2)],
          'osvm__nu': [0.05, 0.1, 0.15, 0.3],
          'osvm__degree': [2, 3]}

In [ ]:
grid_search = GridSearchCV(pipeline, params, cv=5, n_jobs=8, scoring='accuracy')

In [ ]:
grid_search.fit(texts, [1]*len(texts))

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_score_

In [ ]:
THG1L_stmt_ids = [pmid for pmid, reader in worst_words[1][1][0]['foci']]

In [ ]:
THG1L_foci_texts = get_text_content_from_stmt_ids(THG1L_stmt_ids)

In [ ]:
db_texts = list(THG1L_foci_texts[1].values())

In [ ]:
db_texts = [universal_extract_text(t) for t in db_texts if t]

In [ ]:
preds = grid_search.predict(db_texts)

In [ ]:
p = [(x+1)/2 for x in preds]

In [ ]:
sum(p)/len(p)

In [ ]:
db_texts[1]

In [ ]:
F13B_texts = get_text_content_for_gene('F13B')

In [ ]:
grid_search.fit(F13B_texts, [1]*len(F13B_texts))

In [ ]:
grid_search.best_score_

In [ ]:
worst_words[0][0]

In [ ]:
F13B_stmt_ids = [pmid for pmid, reader in worst_words[0][1][0]['fibrin']]

In [ ]:
F13B_db_texts = get_text_content_from_stmt_ids(F13B_stmt_ids)

In [ ]:
F13B_db_texts = list(F13B_db_texts[1].values())

In [ ]:
F13B_db_texts[10]

In [ ]:
F13B_db_texts = [universal_extract_text(t) for t in F13B_db_texts if t]

In [ ]:
preds = grid_search.predict(F13B_db_texts)

In [ ]:
preds = [(x+1)/2 for x in preds]

In [ ]:
sum(preds)/len(preds)

In [ ]:
F13B_db_texts[50]

In [ ]:
list(worst_words[0][1][0].keys())

In [ ]:
list(hgnc_texts['F13B'].keys())

In [ ]:
hgnc_texts['F13B']['F13b']

In [ ]:
insulin_texts = get_text_content_for_gene('INS')

In [ ]:
insulin_texts = [universal_extract_text(text) for text in insulin_texts]

In [ ]:
pipeline.fit(insulin_texts)

In [ ]:
df.head()

In [ ]:
df.head(20)

In [ ]:
INS_stmts = [stmt for stmt, reader in worst_words[15][1][0]['insulin']]

In [ ]:
INS_db_texts = get_text_content_from_stmt_ids(INS_stmts)

In [ ]:
db_texts = [universal_extract_text(t) for t in INS_db_texts[1].values() if t]

In [ ]:
preds = pipeline.predict(db_texts)

In [ ]:
sum([(x+1)/2 for x in preds])/len(preds)

In [ ]:
worst_words[8][1]

In [ ]:
db.session.rollback()

In [ ]:
describe_table('raw_agents')

In [ ]:
describe_table('raw_statements')

In [ ]:
from sqlalchemy import text as sql_text

In [ ]:
query = """
        SELECT stmt.id
        FROM
            raw_statements stmt
        INNER JOIN
            (SELECT * FROM raw_agents ra
             WHERE 
                 ra.db_name = 'TEXT' AND
                 ra.db_id LIKE 'MYL_' AND
                 ra.stmt_id IS NOT NULL) myl
        ON stmt.id = myl.stmt_id
        WHERE
            stmt.type = 'Complex'
        """
myl_statements = list(db.session.execute(query))

In [ ]:
myl_statements = [x[0] for x in myl_statements]

In [ ]:
myl_statements

In [ ]:
query = """
        SELECT db_id 
        FROM
            raw_agents
        WHERE
            stmt_id = ANY(:stmts)
        """
myl_binders = list(db.session.execute(sql_text(query), {'stmts': myl_statements}))

In [ ]:
db.session.rollback()

In [ ]:
myl_binders

In [ ]:
[x for x in worst_words if 'light' in x[1][0]]

In [ ]:
type(hgnc_texts)

In [ ]:
crys = [(x, y) for x, y in hgnc_texts.items() if 'crystallin' in y]

In [ ]:
len(crys)

In [ ]:
crys

In [ ]:
query1 = """SELECT content FROM text_content
           WHERE (text_ref_id, source, text_type) IN ((176393, 'pubmed', 'abstract'),
                                                      (223531, 'manuscripts', 'fulltext'),
                                                      (225473, 'pmc_oa', 'fulltext'))
        """

In [ ]:
res1 = list(db.session.execute(query1))

In [ ]:
unpack(res1[2][0])

In [ ]:
query2 = """SELECT content FROM text_content AS tc
            JOIN (VALUES (176393, 'pubmed', 'abstract'),
                         (223531, 'manuscripts', 'fulltext'),
                         (225473, 'pmc_oa', 'fulltext')) 
            AS ids (text_ref_id, source, text_type)
            ON tc.text_ref_id = ids.text_ref_id AND tc.source = ids.source AND tc.text_type = ids.text_type
         """

In [ ]:
res2 = list(db.session.execute(query2))

In [ ]:
unpack(res2[2][0])

In [ ]:
query3 = """SELECT content FROM text_content AS tc
            JOIN (VALUES :identifiers) 
            AS ids (text_ref_id, source, text_type)
            ON tc.text_ref_id = ids.text_ref_id AND tc.source = ids.source AND tc.text_type = ids.text_type
         """

In [ ]:
res2 = list(db.session.execute(query3, {'identifiers': [[176393, 'pubmed', 'abstract'],
                                                        [223531, 'manuscripts', 'fulltext'],
                                                        [225473, 'pmc_oa', 'fulltext']]}))

In [ ]:
identifiers = [(176393, 'pubmed', 'abstract'), (223531, 'manuscripts', 'fulltext'), (225473, 'pmc_oa', 'fulltext')]
id_str = ', '.join('(:trid%s, :source%s, :text_type%s)' % (i, i, i) for i in range(len(identifiers)))
params = {}
for i, (trid, source, text_type) in enumerate(identifiers):
    params.update({'trid%s' %i: trid, 'source%i' %i: source, 'text_type%i' %i: text_type})

In [ ]:
id_str

In [ ]:
params

In [ ]:
query3 = """SELECT tc.text_ref_id, tc.source, tc.text_type, content FROM text_content AS tc
           JOIN (VALUES %s)
           AS ids (text_ref_id, source, text_type)
           ON tc.text_ref_id = ids.text_ref_id AND tc.source = ids.source AND tc.text_type = ids.text_type
         """ % id_str

In [ ]:
query3

In [ ]:
res3 = list(db.session.execute(query3, params))

In [ ]:
res3

In [ ]:
d

In [ ]:
db.session.rollback()

In [ ]:
res

In [ ]:
describe_table('text_refs')

In [ ]:
from indra_db.util.content_scripts import get_stmts_with_agent_text_like, get_text_content_from_stmt_ids

In [ ]:
from indra_db.util.content_scripts import get_content_identifiers_from_stmt_ids2

In [ ]:
GSC_stmts = get_stmts_with_agent_text_like('GSC')['GSC']

In [ ]:
%time res1 = get_content_identifiers_from_stmt_ids(GSC_stmts)

In [ ]:
%time res2 = get_content_identifiers_from_stmt_ids2(GSC_stmts)

In [ ]:
res2 == res1

In [ ]:
ER_stmts = get_stmts_with_agent_text_like('ER')['ER']

In [ ]:
%time res1 = get_text_content_from_stmt_ids(ER_stmts)

In [ ]:
len(res1)

In [ ]:
%time res2 = get_content_identifiers_from_stmt_ids2(ER_stmts)

In [ ]:
len(res2)

In [ ]:
from sqlalchemy import func

In [ ]:
trid_q = (db.session.query(func.array_agg(db.RawStatements.id)
                               .label('stmt_ids'),
                               db.TextRef.id.label('text_ref_id'))
              .filter(db.RawStatements.id.in_(stmt_ids),
                      *db.link(db.RawStatements, db.TextRef))
              .group_by(db.TextRef.id))
trid_q = trid_q.subquery('text_ref_ids')
texts_q = (db.session.query(db.TextContent.source,
                                db.TextContent.text_type,
                                trid_q)
                 .filter(trid_q.c.text_ref_id ==
                         db.TextContent.text_ref_id))

In [ ]:
from sqlparse import format

In [ ]:
print(format(str(texts_q), reindent=True, keyword_case='upper'))

In [ ]:
def get_content_identifiers_from_stmt_ids2(stmt_ids):
    stmt_ids = tuple(set(stmt_ids))
    query = """SELECT
                rs.id,
                tc.text_ref_id,
                tc.source,
                tc.format,
                tc.text_type
            FROM
                raw statements rs
            JOIN
                reading rd
            ON rs.reading_id = rd.id
            WHERE rs.id IN :stmt_ids
            
                text_content tc
            JOIN
                reading rd
            ON
                rd.text_content_id = tc.id
            JOIN
                text_ref tr
            ON
                tr.id = tc.text_ref_id
            JOIN
                (SELECT
                    raw_statements.id id
                 FROM
                     raw_statements
                 JOIN
                     reading
                 ON
                     raw_statements.reading_id = reading.id
                ) stmts
            ON
            WHERE
                stmts.id IN :stmt_ids        
            """
    res = db.session.execute(query, {'stmt_ids': stmt_ids})
    priority = {'fulltext': 2, 'abstract': 1, 'title': 0}
    seen_text_refs = {}
    ref_dict = {}
    for stmt_id, text_ref_id, source, format_, text_type in res.fetchall():
        new_identifier = (text_ref_id, source, format_, text_type)
        if text_ref_id not in seen_text_refs:
            seen_text_refs[text_ref_id] = new_identifier
            ref_dict[stmt_id] = new_identifier
        else:
            # update if we find text_type with higher priority for
            # a given text_ref
            old_identifier = seen_text_refs[text_ref_id]
            old_text_type = old_identifier[3]
            if priority[text_type] > priority[old_text_type]:
                seen_text_refs[text_ref_id] = new_identifier
                ref_dict[stmt_id] = new_identifier
    return ref_dict

In [ ]:
describe_table('raw_statements')

In [ ]:
describe_table('reading')

In [ ]:
describe_table('text_content')

In [ ]:
show_indices('reading')

In [ ]:
show_indices('raw_stmt_ref_link')

In [ ]:
query = """CREATE INDEX
               text_ref_ix
           ON
               public.raw_stmt_ref_link
           USING
               btree (text_ref_id)
        """

In [ ]:
db.session.execute(query)

In [ ]:
db.session.commit()

In [ ]:
db.session.rollback()

In [ ]:
show_matviews()

In [ ]:
show_indices('raw_stmt_ref_link')

In [ ]:
show_indices('reading')

In [ ]:
show_indices('raw_statements')

In [ ]:
db.session.rollback()

In [ ]:
query = """SHOW temp_file_limit"""

In [ ]:
res = list(db.session.execute(query))

In [ ]:
res

In [ ]:
describe_table('rstmt_ref_link')

In [ ]:
show_matviews()

In [ ]:
views = show_matviews()

In [ ]:
views[-2][4]

In [ ]:
query = """SELECT
                   rs.id, tc.text_ref_id, tc.source, tc.format, tc.text_type
               FROM
                   raw_statements rs
               JOIN
                   raw_stmt_ref_link link
               ON
                   rs.id = link.stmt_id
               JOIN
                   text_ref tr
               ON
                   link.text_ref_id = tr.id
               JOIN text_content tc
                   ON tr.text_content_id tc.id
               WHERE
                   rs.id IN :stmt_ids
            """

In [1]:
from indra_db.util.content_scripts import get_stmts_with_agent_text_like, get_text_content_from_stmt_ids, \
    get_content_identifiers_from_stmt_ids, get_content_identifiers_from_stmt_ids2

In [14]:
stmt_ids = get_stmts_with_agent_text_like('ER')['ER']

In [15]:
%time res = get_text_content_from_stmt_ids(stmt_ids)

CPU times: user 12.3 s, sys: 4.83 s, total: 17.1 s
Wall time: 1min 56s


In [10]:
len(res[1])

27392

In [12]:
%time res = get_content_identifiers_from_stmt_ids(stmt_ids)

CPU times: user 286 ms, sys: 133 ms, total: 419 ms
Wall time: 827 ms


In [13]:
%time res2 = get_content_identifiers_from_stmt_ids2(stmt_ids)

CPU times: user 268 ms, sys: 29.2 ms, total: 297 ms
Wall time: 603 ms


In [ ]:
len(res2)

In [ ]:
x = _collect_content_identifiers(res2)

In [ ]:
len(x)

In [ ]:
len(res1)

In [ ]:
len(x)

In [ ]:
len(stmt_ids)

In [ ]:
x

In [ ]:
res2

In [ ]:
"

In [ ]:
res = db.session.execute(query)

In [ ]:
x

In [24]:
def get_content_identifiers_from_stmt_ids1(stmt_ids, db=None):
    """Get content identifiers for statements from a list of ids

    An identifier is a triple containing a text_ref_id, source, and text_type
    Gets the identifier for best piece of text content with priority
    fulltext > abstract > title

    Parameters
    ----------
    stmt_ids : list of str

    db : Optional[:py:class:`DatabaseManager`]
        User has the option to pass in a database manager. If None
        the primary database is used. Default: None

    Returns
    -------
    ref_dict: dict
        dict mapping statement ids to identifiers for pieces of content.
        These identifiers take the form `<text_ref_id>/<source>/<text_type>'.
        No entries exist for statements with no associated text content
        (these typically come from databases)
    """
    if db is None:
        db = get_primary_db()
    stmt_ids = tuple(set(stmt_ids))
    query = """SELECT
                   sub.stmt_id, tc.text_ref_id, tc.source,
                   tc.format, tc.text_type
               FROM
                   text_content tc,
                   (SELECT
                        stmt_id, text_ref_id
                    FROM
                        raw_stmt_ref_link
                    WHERE
                        stmt_id IN :stmt_ids) sub
                WHERE
                    tc.text_ref_id = sub.text_ref_id
            """
    res = db.session.execute(text(query), {'stmt_ids': stmt_ids})
    return _collect_content_identifiers(res)

In [26]:
%time get_content_identifiers_from_stmt_ids1(stmt_ids)

NameError: name '_collect_content_identifiers' is not defined

In [6]:
cd ~/gilda/models/

/Users/albertsteppi/gilda/models


In [8]:
from learn import get_texts_for_term, get_texts_for_term2, get_pmids

AttributeError: 'str' object has no attribute 'db'